<h1> Pharmaceutical Drug Approval in United States </h1>

---

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
trial_drugbank_df = pd.read_csv('./Datasets/Trial and Drugbank Data.csv')

<h2> 1. Data Cleaning

In [142]:
trial_drugbank_df.shape

(99774, 24)

In [143]:
trial_drugbank_df.isnull().sum()

NCT ID                           0
Trial Status                     0
Last Known Trial Status      97524
Phase                            0
Start Date                       0
Completion Date                  0
Trial Duration (Days)            0
Trial Location                   0
Sponsor Name                     0
Sponsor Type                     0
FDA Regulated Drug           59945
Intervention Name                0
Treated Condition                0
No of Participants            1415
Healthy Participants           139
Gender                          14
Min Age                       4545
Max Age                      54139
Clinical Trial Drug Name        70
First Approval Date          73818
Highest Development Phase    66898
Approval Status              60129
ATC Name                     65867
ATC Class                    65867
dtype: int64

<h3> 1.1. Dropping of nulls, deletion of rows/columns, and changing of datatypes

In [144]:
# Dropping of rows with null values
trial_drugbank_df = trial_drugbank_df.dropna(subset = ['Clinical Trial Drug Name'])
trial_drugbank_df = trial_drugbank_df.dropna(subset = ['No of Participants'])
trial_drugbank_df = trial_drugbank_df.dropna(subset = ['Healthy Participants'])
trial_drugbank_df = trial_drugbank_df.dropna(subset = ['Gender'])

# Dropping of unnecessary columns
trial_drugbank_df = trial_drugbank_df.drop(['Min Age', 'Max Age', 'FDA Regulated Drug', 'Intervention Name', 'Highest Development Phase'], axis = 1)
trial_drugbank_df.isnull().sum()

NCT ID                          0
Trial Status                    0
Last Known Trial Status     95947
Phase                           0
Start Date                      0
Completion Date                 0
Trial Duration (Days)           0
Trial Location                  0
Sponsor Name                    0
Sponsor Type                    0
Treated Condition               0
No of Participants              0
Healthy Participants            0
Gender                          0
Clinical Trial Drug Name        0
First Approval Date         72799
Approval Status             59476
ATC Name                    65082
ATC Class                   65082
dtype: int64

The columns clinical trial drug name, gender, no. of participants and healthy participants are very important for our analysis and prediction models. Any null values should be dropped to avoid causing errors or inaccuracies for the model. Fortunately, there are not that many null values in these columns and removing them will have an insignificant impact on the overall data size. <br>

The columns ATC Name and ATC Class have too many null values and won't be useful for our prediction models. However, the data can still be used for EDA. <br>

The Min and Max Age columns are too unspecific to be of any use for our analysis. The median age of participants might be more useful. <br>

The FDA Regulated Drugs contain too many null values which cannot be assumed to be not FDA regulated drugs, not a useful column.<br>

The intervention name column is a redundant column as it gives the same information as clinical trial drug name. <br>

The Highest Development Phase column does not provide any useful information for the analysis.

In [145]:
mask = (trial_drugbank_df['Approval Status'] != 'approved') & (trial_drugbank_df['First Approval Date'].notnull())
not_null = mask.sum()

print("How many rows are showing first approval dates for 'not approved' drugs?", not_null)

How many rows are showing first approval dates for 'not approved' drugs? 148


In [146]:
mask = (trial_drugbank_df['Approval Status'] != 'approved') 
trial_drugbank_df.loc[mask, 'First Approval Date'] = None

# Check if any such rows exist
mask = (trial_drugbank_df['Approval Status'] != 'approved') & (trial_drugbank_df['First Approval Date'].notnull())
not_null = mask.sum()

print("How many rows are showing first approval dates for 'not approved' drugs?", not_null)

How many rows are showing first approval dates for 'not approved' drugs? 0


There were 148 entry errors under the 'First Approval Date' column where dates were showing even for drugs that were not approved. Not a big number and can be rectified easily.

In [147]:
trial_drugbank_df['Phase'].value_counts()

Phase
PHASE2            36849
PHASE1            28301
PHASE3            18960
PHASE1, PHASE2     9496
EARLY_PHASE1       2397
PHASE2, PHASE3     2185
Name: count, dtype: int64

In [148]:
# Filter out Early phase 1, phase1/2, and phase2/3 clinical trials from dataset.
mask = ~trial_drugbank_df['Phase'].isin(['EARLY_PHASE1', 'PHASE1, PHASE2', 'PHASE2, PHASE3'])
trial_drugbank_df = trial_drugbank_df.loc[mask]
trial_drugbank_df['Approval Status'].value_counts()

Approval Status
approved           28302
investigational     4656
experimental         161
nutraceutical          2
illicit                1
Name: count, dtype: int64

Early phase 1 clinical trials are not done for most drug development, and are considered optional exploratory studies. Therefore, excluded from our dataset.<br>

Phase 1/2 and phase 2/3 clinical trials are not as clearly defined as separate phase 1, 2, and 3 trials and will be difficult to predict using our models. Therefore, excluded from our dataset.

In [149]:
trial_drugbank_df['Approval Status'].value_counts()

Approval Status
approved           28302
investigational     4656
experimental         161
nutraceutical          2
illicit                1
Name: count, dtype: int64

In [150]:
# Filter out nutraceuticals and illicit drugs.
mask = ~trial_drugbank_df['Approval Status'].isin(['illicit', 'nutraceutical'])
trial_drugbank_df = trial_drugbank_df.loc[mask]
trial_drugbank_df['Approval Status'].value_counts()

Approval Status
approved           28302
investigational     4656
experimental         161
Name: count, dtype: int64

Both nutraceuticals and illicit drugs are not eligible for regulatory approval and not useful for our analysis or model.

In [151]:
trial_drugbank_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 84107 entries, 0 to 99773
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   NCT ID                    84107 non-null  object 
 1   Trial Status              84107 non-null  object 
 2   Last Known Trial Status   1640 non-null   object 
 3   Phase                     84107 non-null  object 
 4   Start Date                84107 non-null  object 
 5   Completion Date           84107 non-null  object 
 6   Trial Duration (Days)     84107 non-null  int64  
 7   Trial Location            84107 non-null  object 
 8   Sponsor Name              84107 non-null  object 
 9   Sponsor Type              84107 non-null  object 
 10  Treated Condition         84107 non-null  object 
 11  No of Participants        84107 non-null  float64
 12  Healthy Participants      84107 non-null  object 
 13  Gender                    84107 non-null  object 
 14  Clinical Tr

In [152]:
trial_drugbank_df['Completion Date'] = pd.to_datetime(trial_drugbank_df['Completion Date'])
trial_drugbank_df['Start Date'] = pd.to_datetime(trial_drugbank_df['Start Date'])

Converted completion date and start date columns to datetime datatypes for easier analysis and processing.

<h3> 1.2. Re-grouping of categories in categorical columns

In [153]:
mask =  (trial_drugbank_df['Completion Date'].dt.year <= 2018) & (trial_drugbank_df['Approval Status'] != 'approved') & (trial_drugbank_df['Phase'] == 'PHASE1')
trial_drugbank_df.loc[mask, 'Approval Status'] = 'not approved'

trial_drugbank_df['Approval Status'].value_counts()

Approval Status
approved           28302
not approved       12326
investigational     3721
experimental         138
Name: count, dtype: int64

For clinical trials with completion dates of 2018 or earlier and are still in Phase 1, if the drug tested in the trial has yet to be approved, their status can be safely assumed to be not approved. Since time duration of phase 2 and 3 trials plus FDA approval rarely extends beyond 7.5 years.

In [154]:
mask =  (trial_drugbank_df['Completion Date'].dt.year <= 2020) & (trial_drugbank_df['Approval Status'] != 'approved') & (trial_drugbank_df['Phase'].isin(['PHASE2']))
trial_drugbank_df.loc[mask, 'Approval Status'] = 'not approved'

trial_drugbank_df['Approval Status'].value_counts()

Approval Status
not approved       29193
approved           28302
investigational     2340
experimental          81
Name: count, dtype: int64

For clinical trials with completion dates of 2020 or earlier and are still in Phase 2, if the drug tested in the trial has yet to be approved, their status can be safely assumed to be not approved. Since time duration of phase 3 trials plus FDA approval rarely extends beyond 5 years.

In [155]:
mask =  (trial_drugbank_df['Completion Date'].dt.year <= 2024) & (trial_drugbank_df['Approval Status'] != 'approved') & (trial_drugbank_df['Phase'].isin(['PHASE3']))
trial_drugbank_df.loc[mask, 'Approval Status'] = 'not approved'

trial_drugbank_df['Approval Status'].value_counts()

Approval Status
not approved       41541
approved           28302
investigational     1651
experimental          50
Name: count, dtype: int64

For clinical trials with completion dates of 2024 or earlier and are still in Phase 3, if the drug tested in the trial has yet to be approved, their status can be safely assumed to be not approved. Since the time duration from end of phase 3 to FDA approval rarely extends beyond 1.5 years.

In [156]:
mask = (trial_drugbank_df['Approval Status'] == 'experimental')
trial_drugbank_df.loc[mask, 'Approval Status'] = 'investigational'
trial_drugbank_df['Approval Status'] = trial_drugbank_df['Approval Status'].fillna('investigational')

trial_drugbank_df['Approval Status'].value_counts()

Approval Status
not approved       41541
approved           28302
investigational    14264
Name: count, dtype: int64

Grouped all other trials under investigational, including those with empty cells, no meaningful difference for our analysis between experimental and investigational.

In [157]:
trial_drugbank_df['Trial Status'].value_counts()

Trial Status
COMPLETED                  64173
TERMINATED                 14237
WITHDRAWN                   2205
UNKNOWN                     1640
ACTIVE_NOT_RECRUITING       1188
RECRUITING                   489
SUSPENDED                    110
NOT_YET_RECRUITING            44
ENROLLING_BY_INVITATION       21
Name: count, dtype: int64

In [158]:
trial_drugbank_df.loc[trial_drugbank_df['Trial Status'] == 'UNKNOWN', 'Trial Status'] = trial_drugbank_df['Last Known Trial Status']
trial_drugbank_df = trial_drugbank_df.drop(['Last Known Trial Status'], axis = 1)

trial_drugbank_df['Trial Status'].value_counts()

Trial Status
COMPLETED                  64173
TERMINATED                 14237
WITHDRAWN                   2205
ACTIVE_NOT_RECRUITING       1950
RECRUITING                  1212
NOT_YET_RECRUITING           151
SUSPENDED                    110
ENROLLING_BY_INVITATION       69
Name: count, dtype: int64

For trials with unknown status, fill in with value from the 'last known trial status' column, and drop the column.

In [159]:
mask =  (trial_drugbank_df['Trial Status'].isin(['TERMINATED', 'WITHDRAWN', 'SUSPENDED']))
trial_drugbank_df.loc[mask, 'Trial Status'] = 'NOT COMPLETED'

trial_drugbank_df['Trial Status'].value_counts()

Trial Status
COMPLETED                  64173
NOT COMPLETED              16552
ACTIVE_NOT_RECRUITING       1950
RECRUITING                  1212
NOT_YET_RECRUITING           151
ENROLLING_BY_INVITATION       69
Name: count, dtype: int64

Grouped trials that are terminated, withdrawn, and suspended under a new group 'NOT COMPLETED'. No meaningful difference between terminated and withdrawn for our analysis. <br> 

Suspended trials might resume, however, all these suspended trials are already over their projected completion date.

In [160]:
mask =  (trial_drugbank_df['Trial Status'].isin(['ACTIVE_NOT_RECRUITING', 'RECRUITING', 'NOT_YET_RECRUITING', 'ENROLLING_BY_INVITATION']))
trial_drugbank_df.loc[mask, 'Trial Status'] = 'IN-PROGRESS'

trial_drugbank_df['Trial Status'].value_counts()

Trial Status
COMPLETED        64173
NOT COMPLETED    16552
IN-PROGRESS       3382
Name: count, dtype: int64

Grouped trials that are recruiting, not yet recruiting, actives, or inviting participants under a new group 'IN-PROGRESS'. Separately, these categories are quite small in sample size and don't present a meaningful difference for our analysis.

In [161]:
trial_drugbank_df['Sponsor Type'].value_counts()

Sponsor Type
INDUSTRY     46498
OTHER        29773
NIH           4959
NETWORK       2142
FED            587
INDIV          125
OTHER_GOV       23
Name: count, dtype: int64

In [162]:
trial_drugbank_df.loc[trial_drugbank_df['Sponsor Name'].str.contains('university|school|college', case = False), 'Sponsor Type'] = 'ACADEMIC'

trial_drugbank_df['Sponsor Type'].value_counts()

Sponsor Type
INDUSTRY     46496
OTHER        17677
ACADEMIC     12104
NIH           4959
NETWORK       2142
FED            582
INDIV          125
OTHER_GOV       22
Name: count, dtype: int64

Instead of grouping university sponsors under the 'Other' category, separating them into their own category will be more useful for our analysis and prediction model.

In [163]:
mask = (trial_drugbank_df['Sponsor Type'] == 'OTHER') & (trial_drugbank_df['Sponsor Name'].str.contains('hospital|clinic|medical center', case = False))
trial_drugbank_df.loc[mask, 'Sponsor Type'] = 'HOSPITALS'

trial_drugbank_df['Sponsor Type'].value_counts()

Sponsor Type
INDUSTRY     46496
OTHER        13121
ACADEMIC     12104
NIH           4959
HOSPITALS     4556
NETWORK       2142
FED            582
INDIV          125
OTHER_GOV       22
Name: count, dtype: int64

Instead of grouping university sponsors under the 'Other' category, separating hospitals, clinics, and medical centers under a healthcare institutions category will be more useful for our analysis and prediction model.

In [164]:
mask = (trial_drugbank_df['Sponsor Type'] == 'OTHER') & (trial_drugbank_df['Sponsor Name'].str.contains('cancer', case = False))
trial_drugbank_df.loc[mask, 'Sponsor Type'] = 'CANCER CENTERS'

trial_drugbank_df['Sponsor Type'].value_counts()

Sponsor Type
INDUSTRY          46496
ACADEMIC          12104
CANCER CENTERS     7207
OTHER              5914
NIH                4959
HOSPITALS          4556
NETWORK            2142
FED                 582
INDIV               125
OTHER_GOV            22
Name: count, dtype: int64

Instead of grouping cancer center sponsors under the 'Other' category, separating cancer centers into their own category will be more useful.

In [165]:
trial_drugbank_df.loc[trial_drugbank_df['Sponsor Type'].isin(['INDIV', 'OTHER_GOV']),'Sponsor Type'] = 'OTHER'

trial_drugbank_df['Sponsor Type'].value_counts()

Sponsor Type
INDUSTRY          46496
ACADEMIC          12104
CANCER CENTERS     7207
OTHER              6061
NIH                4959
HOSPITALS          4556
NETWORK            2142
FED                 582
Name: count, dtype: int64

Individuals and other governments are very small categories and don't present a meaningful difference to our analysis. Therefore, grouped under 'Other' category.

In [166]:
top_10_pharma = ['Pfizer', 'Eli Lilly and Company', 'Johnson & Johnson Pharmaceutical Research & Development, L.L.C.', 'Johnson & Johnson Consumer Inc., McNeil Consumer Healthcare Division', 
                 'Johnson & Johnson Consumer and Personal Products Worldwide', 'Johnson & Johnson Consumer Inc. (J&JCI)', 'Johnson & Johnson Healthcare Products Division of McNEIL-PPC, Inc.', 
                 'Johnson & Johnson Vision Care, Inc.', 'Merck Sharp & Dohme LLC', 'Novartis Pharmaceuticals', 'Novartis Vaccines', 'Novartis Gene Therapies', 'Novartis', 'Hoffmann-La Roche', 
                 'Novo Nordisk A/S', 'AstraZeneca', 'Amgen', 'AbbVie']
trial_drugbank_df.loc[trial_drugbank_df['Sponsor Name'].isin(top_10_pharma),'Sponsor Type'] = 'TOP 10 PHARMA'

trial_drugbank_df['Sponsor Type'].value_counts()

Sponsor Type
INDUSTRY          36055
ACADEMIC          12104
TOP 10 PHARMA     10441
CANCER CENTERS     7207
OTHER              6061
NIH                4959
HOSPITALS          4556
NETWORK            2142
FED                 582
Name: count, dtype: int64

Grouped companies that fall into top 10 pharma list by market capitalization into a Top 10 Pharma category. This might be an interesting category for our analysis or prediction model.

In [167]:
trial_drugbank_df['Treated Condition'].value_counts()

Treated Condition
Healthy                                                              2718
Breast Cancer                                                        1373
Healthy Volunteers                                                   1040
Multiple Myeloma                                                      941
Prostate Cancer                                                       939
                                                                     ... 
Transplantation, Liver                                                  1
Pre Diabetes                                                            1
Repeated IVF Failure                                                    1
Alcohol Use Disorder (AUD), Post Traumatic Stress Disorder (PTSD)       1
Cancer, Non-small Cell Lung Cancer (NSCLC)                              1
Name: count, Length: 18260, dtype: int64

In [168]:
keyword_groups = {
    "Cancer": ["cancer", "tumor", "carcinoma", "leukemia", "lymphoma", "sarcoma","neoplasm", "malignancy", "oncology", "melanoma", "myeloma", "blastoma", "gist"],
    "Infectious Disease": ["infection", "hiv", "aids", "covid", "virus", "bacteria", "malaria", "tuberculosis","ebola", "hepatitis", "flu", "influenza", "sepsis", "abscess", "fungal", "parasite"],
    "Cardiovascular": ["heart", "cardio", "hypertension", "stroke", "vascular", "artery","angina", "myocardial", "atherosclerosis", "atrial", "arrhythmia", "cardiomyopathy", "tachycardia"],
    "Metabolic/Endocrine": ["diabetes", "thyroid", "insulin", "metabolic", "obesity","dyslipidemia", "cholesterol", "glucose", "hyperlipidemia", "adrenal", "aldosteronism", "cushing", "pituitary", "diabetic"],
    "Mental Health": ["depression", "anxiety", "bipolar", "schizophrenia", "mental", "ptsd","psychosis", "autism", "adhd", "suicide", "panic", "ocd", "depressive"],
    "Neurological": ["parkinson", "alzheimer", "seizure", "epilepsy", "neuro", "brain","dementia", "multiple sclerosis", "neuropathy", "migraine", "stroke", "neuralgia", "als", "huntington"],
    "Autoimmune": ["lupus", "arthritis", "psoriasis", "autoimmune", "crohn","scleroderma", "celiac", "sjogren", "myositis", "vasculitis", "hashimoto"],
    "Respiratory": ["asthma", "copd", "respiratory", "lung", "bronchitis","emphysema", "pulmonary", "airway", "interstitial"],
    "Digestive": ["liver", "hepatic", "pancreas", "colon", "digestive", "bowel", "gut","gastric", "esophageal", "intestinal", "constipation", "diarrhea", "ulcerative"],
    "Musculoskeletal": ["muscle", "bone", "joint", "osteoporosis", "scoliosis","myopathy", "fibromyalgia", "spine", "orthopedic", "muscular"],
    "Genetic": ["genetic", "syndrome", "mutation", "congenital", "trisomy", "chromosome", "inherited", "deletion", "dysplasia", "monogenic"],
    "Hematological": ["anemia", "hemophilia", "thalassemia", "sickle", "hematologic", "iron deficiency", "blood disorder"],
    "Ocular": ["eye", "vision", "retina", "macula", "macular", "optic", "ocular","glaucoma", "cataract", "retinopathy", "retinitis", "conjunctivitis", "keratitis", "uveitis", "amblyopia", "strabismus", 
               "nystagmus","dry eye", "cornea", "macular edema"],
    "Dermatological": ["skin", "dermatitis", "eczema", "psoriasis", "rash", "acne", "hives", "urticaria", "itch", "itching", "alopecia", "melanoma", "vitiligo", "rosacea", "scleroderma",
                       "cellulitis", "lesion", "cutaneous", "dermato", "blister", "boil", "abscess"],
    "Healthy": ["healthy"],
    "Other": []
    }

In [169]:
# Function which assigns one category only (first match in priority order)
def assign_primary_group(text):
    text = text.lower()
    for category, keywords in keyword_groups.items():
        if any(keyword in text for keyword in keywords):
            return category
    return "Other"

# Apply to dataframe
trial_drugbank_df['Medical Condition'] = trial_drugbank_df['Treated Condition'].apply(assign_primary_group)

trial_drugbank_df['Medical Condition'].value_counts()

Medical Condition
Cancer                 34513
Other                  14696
Infectious Disease      6994
Healthy                 5366
Metabolic/Endocrine     4022
Cardiovascular          2891
Neurological            2804
Autoimmune              2616
Mental Health           2261
Respiratory             1876
Genetic                 1197
Dermatological          1197
Digestive               1154
Ocular                  1049
Hematological            923
Musculoskeletal          548
Name: count, dtype: int64

The treated conditions column contain too many small and highly specific categories which are hard to analyse and identify trend. <br>

Therefore a keyword list was developed to group the conditions treated by the drug into broader categories.

In [170]:
trial_drugbank_df['Clinical Trial Drug Name'].info()

<class 'pandas.core.series.Series'>
Index: 84107 entries, 0 to 99773
Series name: Clinical Trial Drug Name
Non-Null Count  Dtype 
--------------  ----- 
84107 non-null  object
dtypes: object(1)
memory usage: 3.3+ MB


In [171]:
# Remove leading/trailing spaces first
trial_drugbank_df['Clinical Trial Drug Name'] = trial_drugbank_df['Clinical Trial Drug Name'].str.strip()

# Keep only rows where Drug Name contains at least one letter or number
trial_drugbank_df = trial_drugbank_df[trial_drugbank_df['Clinical Trial Drug Name'].str.contains(r'[a-zA-Z0-9]', na=False)]

# Drop rows that contain only numbers
trial_drugbank_df = trial_drugbank_df[~trial_drugbank_df['Clinical Trial Drug Name'].str.fullmatch(r'\d+', na = False)]

trial_drugbank_df['Clinical Trial Drug Name'].info()

<class 'pandas.core.series.Series'>
Index: 84059 entries, 0 to 99773
Series name: Clinical Trial Drug Name
Non-Null Count  Dtype 
--------------  ----- 
84059 non-null  object
dtypes: object(1)
memory usage: 1.3+ MB


Delete away rows where the drug names are incoherent and not useful for analysis.

<h3> 1.3. Creation of new columns and calculated fields

In [172]:
trial_drugbank_df['Drug-Condition'] = trial_drugbank_df['Clinical Trial Drug Name'] + '-' + trial_drugbank_df['Medical Condition']
trial_drugbank_df.rename(columns = {'Clinical Trial Drug Name': 'Clinical Trial Drug'}, inplace = True)
trial_drugbank_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 84059 entries, 0 to 99773
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   NCT ID                 84059 non-null  object        
 1   Trial Status           84059 non-null  object        
 2   Phase                  84059 non-null  object        
 3   Start Date             84059 non-null  datetime64[ns]
 4   Completion Date        84059 non-null  datetime64[ns]
 5   Trial Duration (Days)  84059 non-null  int64         
 6   Trial Location         84059 non-null  object        
 7   Sponsor Name           84059 non-null  object        
 8   Sponsor Type           84059 non-null  object        
 9   Treated Condition      84059 non-null  object        
 10  No of Participants     84059 non-null  float64       
 11  Healthy Participants   84059 non-null  object        
 12  Gender                 84059 non-null  object        
 13  Clinic

A new column was created by joining drug name with the medical conditions these drugs are being tested for, to create a drug-condition pair which might be useful for analysis.

In [173]:
trial_drugbank_df['No. of Trial Locations'] = trial_drugbank_df['Trial Location'].apply(lambda x: len([c.strip() for c in x.split(',')]))
trial_drugbank_df[['Trial Location', 'No. of Trial Locations']]

,Trial Location,No. of Trial Locations
0,United States,1
1,United States,1
2,United States,1
3,United States,1
4,United States,1
...,...,...
99769,United States,1
99770,United States,1
99771,"United States, Australia, Austria, Belgium, Br...",28
99772,"United States, Australia, Austria, Belgium, Br...",28


Created a new column which gives the total unique trial locations (countries) for each clinical trial being conducted.

In [174]:
# Remove duplicates for clean Sponsor-TrialID combinations
unique_trials = trial_drugbank_df[['Sponsor Name', 'NCT ID', 'Start Date', 'Completion Date', 'Trial Status']].drop_duplicates()

# Self-merge on Sponsor
merged = unique_trials.merge(unique_trials, on='Sponsor Name', suffixes=('', '_prior'))

# Filter prior trials to keep different trial IDs and where prior trial completion date < current trial start date 
mask = ((merged['NCT ID'] != merged['NCT ID_prior']) & (merged['Completion Date_prior'] < merged['Start Date']))
prior_trials = merged[mask]

# Count completed prior trials
completed_counts = (prior_trials[prior_trials['Trial Status_prior'] == 'COMPLETED'].groupby(['Sponsor Name', 'NCT ID']).size().reset_index(name = 'Sponsor_Prior Completed Trials'))

# Count not completed prior trials
not_completed_counts = (prior_trials[prior_trials['Trial Status_prior'] == 'NOT COMPLETED'].groupby(['Sponsor Name', 'NCT ID']).size().reset_index(name = 'Sponsor_Prior Not Completed Trials'))

# Merge counts back into the main dataset
trial_drugbank_df = trial_drugbank_df.merge(completed_counts, on = ['Sponsor Name', 'NCT ID'], how = 'left')
trial_drugbank_df = trial_drugbank_df.merge(not_completed_counts, on = ['Sponsor Name', 'NCT ID'], how = 'left')

# Fill null values with 0
trial_drugbank_df['Sponsor_Prior Not Completed Trials'] = trial_drugbank_df['Sponsor_Prior Not Completed Trials'].fillna(0).astype(int)
trial_drugbank_df['Sponsor_Prior Completed Trials'] = trial_drugbank_df['Sponsor_Prior Completed Trials'].fillna(0).astype(int)

trial_drugbank_df.iloc[:, -5:].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84059 entries, 0 to 84058
Data columns (total 5 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   Medical Condition                   84059 non-null  object
 1   Drug-Condition                      84059 non-null  object
 2   No. of Trial Locations              84059 non-null  int64 
 3   Sponsor_Prior Completed Trials      84059 non-null  int32 
 4   Sponsor_Prior Not Completed Trials  84059 non-null  int32 
dtypes: int32(2), int64(1), object(2)
memory usage: 2.6+ MB


Created a new column which aggregates the sponsor's previous track record of completed and non-completed clinical trials.

In [175]:
# Drop duplicates to ensure only unique Drug-NCT ID-Phase combinations are counted
unique_trials = trial_drugbank_df[['Clinical Trial Drug', 'NCT ID', 'Trial Status', 'Phase']].drop_duplicates()

# Create pivot table: counts of unique NCT IDs by Drug, Trial Status, and Phase
trial_counts = (unique_trials.groupby(['Clinical Trial Drug', 'Phase', 'Trial Status'])['NCT ID'].nunique().reset_index(name = 'Trial Count'))

# Pivot to wide format: one row per Drug, columns for each Phase and Status combo
trial_counts_pivot = (trial_counts.pivot_table(index = 'Clinical Trial Drug', columns = ['Phase', 'Trial Status'], values = 'Trial Count', fill_value = 0).reset_index())

# Flatten multi-level columns
trial_counts_pivot.columns = ['Clinical Trial Drug'] + [f"Total {status} {phase} Trials per Drug" for phase, status in trial_counts_pivot.columns.tolist()[1:]]

# Merge back to main dataframe
trial_drugbank_df = trial_drugbank_df.merge(trial_counts_pivot, on = 'Clinical Trial Drug', how='left')

trial_drugbank_df.iloc[:, -15:].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84059 entries, 0 to 84058
Data columns (total 15 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   ATC Class                                   28341 non-null  object 
 1   Medical Condition                           84059 non-null  object 
 2   Drug-Condition                              84059 non-null  object 
 3   No. of Trial Locations                      84059 non-null  int64  
 4   Sponsor_Prior Completed Trials              84059 non-null  int32  
 5   Sponsor_Prior Not Completed Trials          84059 non-null  int32  
 6   Total COMPLETED PHASE1 Trials per Drug      84059 non-null  float64
 7   Total IN-PROGRESS PHASE1 Trials per Drug    84059 non-null  float64
 8   Total NOT COMPLETED PHASE1 Trials per Drug  84059 non-null  float64
 9   Total COMPLETED PHASE2 Trials per Drug      84059 non-null  float64
 10  Total IN-P

Created 15 new columns which aggregates for each drug, the total number of completed, not completed, and in-progress clinical trials being conducted, for each trial phase.

In [176]:
# Drop duplicates to avoid double-counting the same trial
unique_trials = trial_drugbank_df[['Clinical Trial Drug', 'Sponsor Type', 'NCT ID']].drop_duplicates()

# Group by Drug and Sponsor Type, then count unique trials
sponsor_type_breakdown = (unique_trials.groupby(['Clinical Trial Drug', 'Sponsor Type'])['NCT ID'].nunique().reset_index(name = 'Trial Count'))

# Pivot the table to have Sponsor Types as columns
sponsor_type_pivot = (sponsor_type_breakdown.pivot_table(index = 'Clinical Trial Drug', columns = 'Sponsor Type', values = 'Trial Count', fill_value = 0).reset_index())

# Rename columns to indicate they are sponsor type counts
sponsor_type_pivot.columns = ['Clinical Trial Drug'] + [f"Total {sponsor_type} Sponsors per Drug" for sponsor_type in sponsor_type_pivot.columns[1:]]

# Merge back to original dataframe
trial_drugbank_df = trial_drugbank_df.merge(sponsor_type_pivot, on = 'Clinical Trial Drug', how = 'left')

Created 9 new columns which aggregates for each drug, the total number of academic, cancer center, FED, healthcare, industry, network, NIH, top 10 pharma, and other sponsors, for clinical trials being conducted.

In [177]:
# Calculate unique medical conditions tested per drug
unique_drug_condition_counts = (trial_drugbank_df.groupby('Clinical Trial Drug')['Medical Condition'].nunique().reset_index(name='No. of Medical Conditions Tested per Drug'))

# Merge counts back into the main dataset
trial_drugbank_df = trial_drugbank_df.merge(unique_drug_condition_counts, on='Clinical Trial Drug', how='left')

trial_drugbank_df.iloc[:, -10:].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84059 entries, 0 to 84058
Data columns (total 10 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Total ACADEMIC Sponsors per Drug           84059 non-null  float64
 1   Total CANCER CENTERS Sponsors per Drug     84059 non-null  float64
 2   Total FED Sponsors per Drug                84059 non-null  float64
 3   Total HOSPITALS Sponsors per Drug          84059 non-null  float64
 4   Total INDUSTRY Sponsors per Drug           84059 non-null  float64
 5   Total NETWORK Sponsors per Drug            84059 non-null  float64
 6   Total NIH Sponsors per Drug                84059 non-null  float64
 7   Total OTHER Sponsors per Drug              84059 non-null  float64
 8   Total TOP 10 PHARMA Sponsors per Drug      84059 non-null  float64
 9   No. of Medical Conditions Tested per Drug  84059 non-null  int64  
dtypes: float64(9), int64(1

In [178]:
unseen_trial_drugbank_df = trial_drugbank_df[trial_drugbank_df['Approval Status'] == 'investigational']
unseen_trial_drugbank_df['Approval Status'].value_counts()

Approval Status
investigational    14261
Name: count, dtype: int64

The dataset contains clinical trial data that are still under investigation whose approval status is unknown. They will be used as unseen data to test our prediction model.

In [179]:
trial_drugbank_df2 = trial_drugbank_df.copy()

In [180]:
trial_drugbank_df = trial_drugbank_df[trial_drugbank_df['Approval Status'] != 'investigational']
trial_drugbank_df['Approval Status'].value_counts()

Approval Status
not approved    41496
approved        28302
Name: count, dtype: int64

Only data with a clear approved and not approved status will be used for our analysis and training of the prediction model.

<h2> 2. Exploratory Data Analysis

<h3> 2.1. Overview of clinical trial data

In [181]:
# Drop duplicates to ensure unique trial per NCT ID
unique_trials = trial_drugbank_df.drop_duplicates(subset = 'NCT ID')

In [182]:
unique_trials['Completion Year'] = unique_trials['Completion Date'].dt.year

count_phase_year = unique_trials.groupby(['Completion Year', 'Phase'])['NCT ID'].count().reset_index(name = 'No. of Trials')
count_phase_year

C:\Users\brage\AppData\Local\Temp\ipykernel_28808\2170149133.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Completion Year,Phase,No. of Trials
0,2003,PHASE1,116
1,2003,PHASE2,205
2,2003,PHASE3,103
3,2004,PHASE1,137
4,2004,PHASE2,215
...,...,...,...
64,2024,PHASE2,492
65,2024,PHASE3,585
66,2025,PHASE1,181
67,2025,PHASE2,267


In [183]:
fig1 = px.line(count_phase_year,
              x = 'Completion Year',
              y = 'No. of Trials',
              color = 'Phase',
              width = 750, 
              height = 500)
fig1.update_layout(title = {'text' : 'Number of Trials Conducted per Phase over Time', 'x' : 0.5, 'xanchor' : 'center'}, xaxis_title = 'Year', yaxis_title_text = 'No. of Trials')
fig1.update_traces(hovertemplate ='%{x}<br>Number of Trials: %{y}<br><extra></extra>')
fig1.show()

The number of completed clinical trials across all phases declined significantly in the United States due to Covid-19 from 2020 onwards. Numbers are yet to recover.

In [184]:
count_phase_status = unique_trials.groupby(['Trial Status', 'Phase'])['NCT ID'].count().reset_index(name = 'No. of Trials').sort_values(by='Trial Status')
count_phase_status

,Trial Status,Phase,No. of Trials
0,COMPLETED,PHASE1,9875
1,COMPLETED,PHASE2,12764
2,COMPLETED,PHASE3,8516
3,IN-PROGRESS,PHASE1,252
4,IN-PROGRESS,PHASE2,588
5,IN-PROGRESS,PHASE3,293
6,NOT COMPLETED,PHASE1,1841
7,NOT COMPLETED,PHASE2,4122
8,NOT COMPLETED,PHASE3,1790


In [185]:
fig2 = px.bar(count_phase_status,
              x = 'Phase',
              y = 'No. of Trials',
              color = 'Trial Status',
              width = 600, 
              height = 500)
fig2.update_layout(title = {'text' : 'No. of Clinical Trials Conducted by Phase', 'x' : 0.5, 'xanchor' : 'center'}, xaxis_title = '', yaxis_title_text='No. of Trials')
fig2.update_traces(hovertemplate ='%{x}<br>No. of Trials: %{y}<br><extra></extra>')
fig2.show()

Phase 1 trials are designed to test the safety of a new drug and determine the correct dosage range. <br>
Phase 2 trials are designed to evaluate the efficacy of a drug in treating specific conditions and its side effects on patients.<br>
Phase 1, phase 2 trials are trials which evaluate both safety and early signs of efficacy within the same clinical study. <br>
Phase 3 trials are carried out a much larger scale to confirm the drug’s effectiveness, monitor side effects, and compare it to standard or placebo treatments.
Phase 2, phase 3 trials are designed to seamlessly transition from exploring efficacy to confirming it on a larger scale within the same clinical study. <br>

There are more phase 2 trials conducted than both phase 1 and 3 trials. This is because typically only one Phase 1 trial is needed per drug. Once safety is established, researchers conduct several Phase 2 studies to explore how well the drug works across different conditions. <br>

The attrition rate is also much higher for phase 2 trials, therefore more trials are carried out at this stage to improve their chances of success.


<h3> 2.2. Effect of clinical trial phase on trial duration

In [186]:
# Group by phase and compute median trial duration
median_duration_phase = unique_trials.groupby('Phase')['Trial Duration (Days)'].median().div(365.25).reset_index(name = 'Median Trial Duration (Years)').sort_values(by='Phase')
median_duration_phase

,Phase,Median Trial Duration (Years)
0,PHASE1,1.754962
1,PHASE2,2.830938
2,PHASE3,2.332649


In [187]:
fig3 = px.bar(median_duration_phase,
              x = 'Phase',
              y = 'Median Trial Duration (Years)',
              width = 500, 
              height = 500)
fig3.update_layout(title = {'text' : 'Median Trial Duration by Phase', 'x' : 0.5, 'xanchor' : 'center'}, xaxis_title = '', yaxis_title_text='Median Trial Duration (Years)')
fig3.update_traces(hovertemplate ='%{x}<br>Median Trial Duration: %{y:.2f} yr<br><extra></extra>')
fig3.show()

Surprisingly, the median trial duration of phase 2 trials are longer than phase 3 trials, despite phase 3 involving more participants are higher complexity. <br>

Phase 2 studies often explore multiple doses, subgroups, or endpoints, which can prolong trial design, recruitment, and analysis. <br>

Phase 3 trials usually have more funding, more sites, and stricter timelines due to commercial and regulatory pressure.

In [188]:
phase1 = unique_trials[unique_trials['Phase'] == 'PHASE1']
phase2 = unique_trials[unique_trials['Phase'] == 'PHASE2']
phase3 = unique_trials[unique_trials['Phase'] == 'PHASE3']

In [189]:
# Create subplots of histograms for each clinical trial phase and combine them in a single plot to investigate the spread of trial duration
fig4 = make_subplots(rows = 1, cols = 3,
                     subplot_titles=('Phase 1 Clinical Trials', 'Phase 2 Clinical Trials', 'Phase 3 Clinical Trials', 'Phase 1/2 Clinical Trials', 'Phase 2/3 Clinical Trials'), 
                     vertical_spacing = 0.1, horizontal_spacing = 0.03)

fig4.add_trace(go.Histogram(x = phase1['Trial Duration (Days)'] / 365.25, name = 'Phase 1', 
                      hovertemplate = 'No. of Clinical Trials: %{y} <br>Trial Duration: %{x:.2f} yr<extra></extra>'), row = 1, col = 1)
fig4.add_trace(go.Histogram(x = phase2['Trial Duration (Days)'] / 365.25, name = 'Phase 2', 
                      hovertemplate = 'No. of Clinical Trials: %{y} <br>Trial Duration: %{x:.2f} yr<extra></extra>'), row = 1, col = 2)
fig4.add_trace(go.Histogram(x = phase3['Trial Duration (Days)'] / 365.25, name = 'Phase 3', 
                      hovertemplate = 'No. of Clinical Trials: %{y} <br>Trial Duration: %{x:.2f} yr<extra></extra>'), row = 1, col = 3)

fig4.add_annotation(text = 'No. of Clinical Trials', xref = 'paper', yref = 'paper', x = -0.047, y = 0.5, showarrow = False, textangle = -90, font = dict(size = 20))  
fig4.update_layout(title = {'text' : 'Spread of Trial Duration for Phase 1, 2, and 3', 'y' : 0.98, 'x' : 0.5, 'xanchor' : 'center', 'yanchor' : 'top', 'font' : {'size' : 24}}, 
                   margin = dict(l = 80, r = 40, t = 80, b = 50), showlegend = False, height = 400)

For each trial phase, the majority of clinical trials have a duration of between 0 to 5 years, except for phase 1 which is significantly faster. <br>

However, a huge spread of the trial duration was observed for all phases up to 25 or even 30 years.

In [190]:
# Create subplots of box plots for each clinical trial phase and combine them in a single plot to investigate the trial duration outliers
fig5 = make_subplots(rows = 1, cols = 3,
                     subplot_titles=('Phase 1 Clinical Trials', 'Phase 2 Clinical Trials', 'Phase 3 Clinical Trials'), 
                     vertical_spacing = 0.1, horizontal_spacing = 0.06)

fig5.add_trace(go.Box(y = phase1['Trial Duration (Days)'] / 365.25, name = 'Phase 1', hovertemplate = 'Trial Duration: %{y:.2f} yr<br><extra></extra>'), row = 1, col = 1)
fig5.add_trace(go.Box(y = phase2['Trial Duration (Days)'] / 365.25, name = 'Phase 2', hovertemplate = 'Trial Duration: %{y:.2f} yr<br><extra></extra>'), row = 1, col = 2)
fig5.add_trace(go.Box(y = phase3['Trial Duration (Days)'] / 365.25, name = 'Phase 3', hovertemplate = 'Trial Duration: %{y:.2f} yr<br><extra></extra>'), row = 1, col = 3)   

fig5.add_annotation(text = 'Trial Duration (Years)', xref = 'paper', yref = 'paper', x = -0.1, y = 0.5, showarrow = False, textangle = -90, font = dict(size = 20))  
fig5.update_layout(title = {'text' : 'Trial Duration Outliers for Phase 1, 2, and 3', 'y' : 0.98, 'x' : 0.5, 'xanchor' : 'center', 'yanchor' : 'top', 'font' : {'size' : 24}}, 
                   margin = dict(l = 80, r = 40, t = 80, b = 50), xaxis1_title = '', showlegend = False, width = 750, height = 600)
fig5.update_xaxes(showticklabels = False, row = 1, col = 1)
fig5.update_xaxes(showticklabels = False, row = 1, col = 2)
fig5.update_xaxes(showticklabels = False, row = 1, col = 3)

A significant number of outliers was observed for every single trial phase. To achieve a better performing prediction model, the data outliers will need to be filtered out.

<h3> 2.3. Effect of various trial features on clinical trial duration

In [191]:
# Group by the various trial variables and compute median trial duration
median_duration_sponsortype = unique_trials.groupby('Sponsor Type')['Trial Duration (Days)'].median().div(365.25).reset_index(name = 'Median Trial Duration (Years)').sort_values(by='Median Trial Duration (Years)')
median_duration_condition = unique_trials.groupby('Medical Condition')['Trial Duration (Days)'].median().div(365.25).reset_index(name = 'Median Trial Duration (Years)').sort_values(by='Median Trial Duration (Years)')
median_duration_gender = unique_trials.groupby('Gender')['Trial Duration (Days)'].median().div(365.25).reset_index(name = 'Median Trial Duration (Years)').sort_values(by='Median Trial Duration (Years)')
median_duration_locations = unique_trials.groupby('No. of Trial Locations')['Trial Duration (Days)'].median().div(365.25).reset_index(name = 'Median Trial Duration (Years)').sort_values(by='Median Trial Duration (Years)')
median_duration_sponsor_complete = unique_trials.groupby('Sponsor_Prior Completed Trials')['Trial Duration (Days)'].median().div(365.25).reset_index(name = 'Median Trial Duration (Years)').sort_values(by='Median Trial Duration (Years)')
median_duration_sponsor_notcomplete = unique_trials.groupby('Sponsor_Prior Not Completed Trials')['Trial Duration (Days)'].median().div(365.25).reset_index(name = 'Median Trial Duration (Years)').sort_values(by='Median Trial Duration (Years)')
median_duration_status = unique_trials.groupby('Trial Status')['Trial Duration (Days)'].median().div(365.25).reset_index(name = 'Median Trial Duration (Years)').sort_values(by='Median Trial Duration (Years)')
median_duration_health = unique_trials.groupby('Healthy Participants')['Trial Duration (Days)'].median().div(365.25).reset_index(name = 'Median Trial Duration (Years)').sort_values(by='Median Trial Duration (Years)')

# Create larger bins for the no. of participants feature
bins = [0, 50, 100, 250, 500, 750, 1000, 2500, 5000, 7500, 10000, float('inf')]
labels = ['0-50', '51-100', '101-250', '251-500', '501-750', '751-1000', '1001-2500', '2501-5000', '5001-7500', '7501-10000','10000+']
unique_trials['Participant Bin'] = pd.cut(unique_trials['No of Participants'], bins=bins, labels=labels)

median_duration_participants = unique_trials.groupby('Participant Bin')['Trial Duration (Days)'].median().div(365.25).reset_index(name='Median Trial Duration (Years)').sort_values(by='Median Trial Duration (Years)')


C:\Users\brage\AppData\Local\Temp\ipykernel_28808\2959280324.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\brage\AppData\Local\Temp\ipykernel_28808\2959280324.py:16: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [192]:
# Create subplots of bar charts for each feature and combine them in a single plot to investigate median trial duration
fig6 = make_subplots(rows = 3, cols = 3, subplot_titles=('Sponsor Type', 'Sponsor Track Record_Completed Trials', 'Sponsor Track Record_Not Completed Trials', 
                                                         'Medical Condition', 'Number of Trial Locations', 'Trial Status', 
                                                         'Gender', 'Number of Participants', 'Healthy Participants'), 
                                                         vertical_spacing = 0.1, horizontal_spacing = 0.03)

fig6.add_trace(go.Bar(x = median_duration_sponsortype['Sponsor Type'], y = median_duration_sponsortype['Median Trial Duration (Years)'], 
                      hovertemplate = 'Sponsor Type: %{x} <br>Median Trial Duration: %{y:.2f} yr<extra></extra>'), row = 1, col = 1)
fig6.add_trace(go.Bar(x = median_duration_sponsor_complete['Sponsor_Prior Completed Trials'], y = median_duration_sponsor_complete['Median Trial Duration (Years)'], 
                      hovertemplate = 'Completed Trials: %{x} <br>Median Trial Duration: %{y:.2f} yr<extra></extra>'), row = 1, col = 2)
fig6.add_trace(go.Bar(x = median_duration_sponsor_notcomplete['Sponsor_Prior Not Completed Trials'], y = median_duration_sponsor_notcomplete['Median Trial Duration (Years)'], 
                      hovertemplate = 'Not Completed Trials: %{x} <br>Median Trial Duration: %{y:.2f} yr<extra></extra>'), row = 1, col = 3)   
fig6.add_trace(go.Bar(x = median_duration_condition['Medical Condition'], y = median_duration_condition['Median Trial Duration (Years)'], 
                      hovertemplate = 'Medical Condition: %{x} <br>Median Trial Duration: %{y:.2f} yr<extra></extra>'), row = 2, col = 1)
fig6.add_trace(go.Bar(x = median_duration_locations['No. of Trial Locations'], y = median_duration_locations['Median Trial Duration (Years)'], 
                      hovertemplate = 'No. of Trial Locations: %{x} <br>Median Trial Duration: %{y:.2f} yr<extra></extra>'), row = 2, col = 2)
fig6.add_trace(go.Bar(x = median_duration_status['Trial Status'], y = median_duration_status['Median Trial Duration (Years)'], 
                      hovertemplate = 'Trial Status: %{x} <br>Median Trial Duration: %{y:.2f} yr<extra></extra>'), row = 2, col = 3)
fig6.add_trace(go.Bar(x = median_duration_gender['Gender'], y = median_duration_gender['Median Trial Duration (Years)'], 
                      hovertemplate = 'Gender: %{x} <br>Median Trial Duration: %{y:.2f} yr<extra></extra>'), row = 3, col = 1)
fig6.add_trace(go.Bar(x = median_duration_participants['Participant Bin'], y = median_duration_participants['Median Trial Duration (Years)'], 
                      hovertemplate = 'No of Participants: %{x} <br>Median Trial Duration: %{y:.2f} yr<extra></extra>'), row = 3, col = 2)
fig6.add_trace(go.Bar(x = median_duration_health['Healthy Participants'], y = median_duration_health['Median Trial Duration (Years)'], 
                      hovertemplate = 'Healthy Participants: %{x} <br>Median Trial Duration: %{y:.2f} yr<extra></extra>'), row = 3, col = 3)

fig6.add_annotation(text = 'Median Trial Duration (Years)', xref = 'paper', yref = 'paper', x = -0.047, y = 0.5, showarrow = False, textangle = -90, font = dict(size = 20))  
fig6.update_layout(title = {'text' : 'Median Trial Duration for Different Trial Variables', 'y' : 0.98, 'x' : 0.5, 'xanchor' : 'center', 'yanchor' : 'top', 'font' : {'size' : 24}}, 
                   margin = dict(l = 80, r = 40, t = 80, b = 50), showlegend = False, height = 1200)

A positive correlation was observed between number of participants and trial duration. For categorical features, the highest trial duration was observed for trials for cancer drugs and network sponsor types.

<h3> 2.4. Overview of drug approval data

In [193]:
# Drop duplicates to ensure only unique drugs are analyzed
unique_drugs = trial_drugbank_df.drop_duplicates(subset = ['Clinical Trial Drug'])

In [194]:
count_approveddrugs_year = unique_drugs.groupby('First Approval Date')['Clinical Trial Drug'].count().reset_index(name = 'No. of Drugs Approved')
count_approveddrugs_year

,First Approval Date,No. of Drugs Approved
0,1940.0,2
1,1941.0,1
2,1942.0,1
3,1943.0,1
4,1946.0,1
...,...,...
77,2019.0,32
78,2020.0,34
79,2021.0,37
80,2022.0,29


In [195]:
fig7 = px.scatter(count_approveddrugs_year,
              x = 'First Approval Date',
              y = 'No. of Drugs Approved',
              range_x=[2005, 2023],
              range_y=[15, 43],
              width = 750, 
              height = 500)
fig7.update_layout(title = {'text' : 'Number of Drugs Approved over Time', 'x' : 0.5, 'xanchor' : 'center'}, xaxis_title = 'Year of First Approval', yaxis_title_text = 'No. of Drugs Approved')
fig7.update_traces(mode = 'lines+markers', line = dict(color = 'blue'), marker = dict(color = 'red'), hovertemplate ='%{x}<br>No. of Drugs Approved: %{y}<br><extra></extra>')
fig7.show()

The number of drug approvals has increased steadily over time. 1996 had the highest new drug approvals of 44 from US-based clinical trials. <br>

This was due to the implementation of the Prescription Drug User Fee Act (PDUFA) in 1992 which allowed the FDA to collect fees from pharmaceutical companies to fund and speed up the drug review process, dramatically reducing review times. <br>

Many drugs that had been under review or delayed in earlier years were approved in bulk post-PDUFA, creating an artificial spike.

In [196]:
filtered_unique_drugs = unique_drugs[unique_drugs['Approval Status'] == 'approved']

count_approveddrugs_class = filtered_unique_drugs.groupby('ATC Name')['Clinical Trial Drug'].count().reset_index(name = 'No. of Drugs Approved').sort_values(by = 'No. of Drugs Approved')
count_approveddrugs_class

,ATC Name,No. of Drugs Approved
3,"ANTIPARASITIC PRODUCTS, INSECTICIDES AND REPEL...",28
12,"SYSTEMIC HORMONAL PREPARATIONS, EXCL. SEX HORM...",36
13,VARIOUS,46
8,MUSCULO-SKELETAL SYSTEM,50
6,DERMATOLOGICALS,53
11,SENSORY ORGANS,53
10,RESPIRATORY SYSTEM,54
4,BLOOD AND BLOOD FORMING ORGANS,64
7,GENITO URINARY SYSTEM AND SEX HORMONES,74
5,CARDIOVASCULAR SYSTEM,122


In [197]:
fig8 = px.bar(count_approveddrugs_class,
              y = 'ATC Name',
              x = 'No. of Drugs Approved',
              width = 750, 
              height = 500)
fig8.update_layout(title = {'text' : 'Number of Drugs Approved by Medical Condition', 'x' : 0.5, 'xanchor' : 'center'}, xaxis_title = 'No. of Drugs Approved', yaxis_title_text = '')
fig8.update_traces(hovertemplate ='%{x}<br>No. of Drugs Approved: %{y}<br><extra></extra>')
fig8.update_yaxes(tickmode='array',
                 tickvals=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
                 ticktext=['Antiparasites', 'Hormones', 'Others', 'Bones and Muscles', 'Eyes & Ears', 'Skin', 'Lungs', 'Blood', 'Reproductive System', 'Heart', 
                           'Disgestive System', 'Antibiotics & Antivirals', 'Brain & Nervous System', 'Cancer & Immune System'])
fig8.show()

The highest number of drug approvals came from cancer-related and brain or nervous system related drugs.

In [198]:
# Drop duplicates to ensure only unique drugs are analyzed
unique_drugs2 = trial_drugbank_df2.drop_duplicates(subset = ['Clinical Trial Drug'])

count_drugs_status = unique_drugs2.groupby('Approval Status')['Clinical Trial Drug'].count().reset_index(name = 'No. of Unique Drugs').sort_values(by = 'No. of Unique Drugs')
count_drugs_status

,Approval Status,No. of Unique Drugs
0,approved,1494
1,investigational,8439
2,not approved,21407


In [199]:
fig9 = px.bar(count_drugs_status,
              x = 'Approval Status',
              y = 'No. of Unique Drugs',
              width = 500, 
              height = 500)
fig9.update_layout(title = {'text' : 'Approval Status of Unique Drugs', 'x' : 0.5, 'xanchor' : 'center'}, xaxis_title = '', yaxis_title_text = 'No. of Unique Drugs')
fig9.update_traces(hovertemplate ='%{x}<br>No. of Unique Drugs: %{y}<br><extra></extra>')
fig9.show()

The overall drug approval rate stands at 4.5% if we include drugs that are currently still under investigation and 6.2% if we exclude that category. <br>

I will be partitioning out drugs still under investigation and using that portion of the data as unseen data for our prediction models.

<h3> 2.5. Effect of various features on drug approval rates

In [200]:
# Create a binary column: 1 if approved, 0 if not approved
unique_drugs['Is Approved'] = (unique_drugs['Approval Status'] == 'approved').astype(bool)

# Group by the no. of completed, not completed, and in-progress phase 1, 2, and 3 trials per drug and compute the drug approval rate
approval_rate_phase1_completed = (unique_drugs.groupby('Total COMPLETED PHASE1 Trials per Drug')['Is Approved'].mean().reset_index(name = 'Approval Rate'))
approval_rate_phase1_notcompleted = (unique_drugs.groupby('Total NOT COMPLETED PHASE1 Trials per Drug')['Is Approved'].mean().reset_index(name = 'Approval Rate'))
approval_rate_phase1_inprogress = (unique_drugs.groupby('Total IN-PROGRESS PHASE1 Trials per Drug')['Is Approved'].mean().reset_index(name = 'Approval Rate'))
approval_rate_phase2_completed = (unique_drugs.groupby('Total COMPLETED PHASE2 Trials per Drug')['Is Approved'].mean().reset_index(name = 'Approval Rate'))
approval_rate_phase2_notcompleted = (unique_drugs.groupby('Total NOT COMPLETED PHASE2 Trials per Drug')['Is Approved'].mean().reset_index(name = 'Approval Rate'))
approval_rate_phase2_inprogress = (unique_drugs.groupby('Total IN-PROGRESS PHASE2 Trials per Drug')['Is Approved'].mean().reset_index(name = 'Approval Rate'))
approval_rate_phase3_completed = (unique_drugs.groupby('Total COMPLETED PHASE3 Trials per Drug')['Is Approved'].mean().reset_index(name = 'Approval Rate'))
approval_rate_phase3_notcompleted = (unique_drugs.groupby('Total NOT COMPLETED PHASE3 Trials per Drug')['Is Approved'].mean().reset_index(name = 'Approval Rate'))
approval_rate_phase3_inprogress = (unique_drugs.groupby('Total IN-PROGRESS PHASE3 Trials per Drug')['Is Approved'].mean().reset_index(name = 'Approval Rate'))

C:\Users\brage\AppData\Local\Temp\ipykernel_28808\72822568.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [201]:
# Create subplots of bar charts for each feature and combine them in a single plot to investigate the drug approval rate
fig10 = make_subplots(rows = 3, cols = 3, subplot_titles=('Number of Completed Phase 1 Trials per Drug', 'Number of Not Completed Phase 1 Trials per Drug', 'Number of In-Progress Phase 1 Trials per Drug', 
                                                         'Number of Completed Phase 2 Trials per Drug', 'Number of Not Completed Phase 2 Trials per Drug', 'Number of In-Progress Phase 2 Trials per Drug',
                                                         'Number of Completed Phase 3 Trials per Drug', 'Number of Not Completed Phase 3 Trials per Drug', 'Number of In-Progress Phase 3 Trials per Drug'), 
                                                         vertical_spacing = 0.05, horizontal_spacing = 0.03)

fig10.add_trace(go.Bar(x = approval_rate_phase1_completed['Total COMPLETED PHASE1 Trials per Drug'], y = approval_rate_phase1_completed['Approval Rate'], 
                      hovertemplate = 'No. of Completed Phase 1 Trials: %{x} <br>Approval Rate: %{y:.2f} <extra></extra>'), row = 1, col = 1)
fig10.add_trace(go.Bar(x = approval_rate_phase1_notcompleted['Total NOT COMPLETED PHASE1 Trials per Drug'], y = approval_rate_phase1_completed['Approval Rate'], 
                      hovertemplate = 'No. of Not Completed Phase 1 Trials: %{x} <br>Approval Rate: %{y:.2f} <extra></extra>'), row = 1, col = 2)
fig10.add_trace(go.Bar(x = approval_rate_phase1_inprogress['Total IN-PROGRESS PHASE1 Trials per Drug'], y = approval_rate_phase1_completed['Approval Rate'], 
                      hovertemplate = 'No. of In-Progress Phase 1 Trials: %{x} <br>Approval Rate: %{y:.2f} <extra></extra>'), row = 1, col = 3)   
fig10.add_trace(go.Bar(x = approval_rate_phase2_completed['Total COMPLETED PHASE2 Trials per Drug'], y = approval_rate_phase2_completed['Approval Rate'], 
                      hovertemplate = 'No. of Completed Phase 2 Trials: %{x} <br>Approval Rate: %{y:.2f} <extra></extra>'), row = 2, col = 1)
fig10.add_trace(go.Bar(x = approval_rate_phase2_notcompleted['Total NOT COMPLETED PHASE2 Trials per Drug'], y = approval_rate_phase2_completed['Approval Rate'], 
                      hovertemplate = 'No. of Not Completed Phase 2 Trials: %{x} <br>Approval Rate: %{y:.2f} <extra></extra>'), row = 2, col = 2)
fig10.add_trace(go.Bar(x = approval_rate_phase2_inprogress['Total IN-PROGRESS PHASE2 Trials per Drug'], y = approval_rate_phase2_completed['Approval Rate'], 
                      hovertemplate = 'No. of In-Progress Phase 2 Trials: %{x} <br>Approval Rate: %{y:.2f} <extra></extra>'), row = 2, col = 3)
fig10.add_trace(go.Bar(x = approval_rate_phase3_completed['Total COMPLETED PHASE3 Trials per Drug'], y = approval_rate_phase3_completed['Approval Rate'], 
                      hovertemplate = 'No. of Completed Phase 3 Trials: %{x} <br>Approval Rate: %{y:.2f} <extra></extra>'), row = 3, col = 1)
fig10.add_trace(go.Bar(x = approval_rate_phase3_notcompleted['Total NOT COMPLETED PHASE3 Trials per Drug'], y = approval_rate_phase3_completed['Approval Rate'], 
                      hovertemplate = 'No. of Not Completed Phase 3 Trials: %{x} <br>Approval Rate: %{y:.2f} <extra></extra>'), row = 3, col = 2)
fig10.add_trace(go.Bar(x = approval_rate_phase3_inprogress['Total IN-PROGRESS PHASE3 Trials per Drug'], y = approval_rate_phase3_completed['Approval Rate'], 
                      hovertemplate = 'No. of In-Progress Phase 3 Trials: %{x} <br>Approval Rate: %{y:.2f} <extra></extra>'), row = 3, col = 3) 

fig10.add_annotation(text = 'Drug Approval Rate', xref = 'paper', yref = 'paper', x = -0.047, y = 0.5, showarrow = False, textangle = -90, font = dict(size = 20))  
fig10.update_layout(title = {'text' : 'Drug Approval Rates for Number of Phase 1, 2, and 3 Trials', 'y' : 0.99, 'x' : 0.5, 'xanchor' : 'center', 'yanchor' : 'top', 'font' : {'size' : 24}}, 
                   margin = dict(l = 80, r = 40, t = 80, b = 50), showlegend = False, height = 1000)
fig10.show()

A clear positive correlation was observed between drug approval rates and the number of in-progress trials for phase 1, 1/2, 2/3, and 3. The same for the number of not completed and completed phase2/3 trials.

In [203]:
# Group by the no. of completed, not completed, and in-progress phase 1, 2, and 3 trials per drug and compute the drug approval rate
approval_rate_academic_sponsors = (unique_drugs.groupby('Total ACADEMIC Sponsors per Drug')['Is Approved'].mean().reset_index(name = 'Approval Rate'))
approval_rate_cancercenter_sponsors = (unique_drugs.groupby('Total CANCER CENTERS Sponsors per Drug')['Is Approved'].mean().reset_index(name = 'Approval Rate'))
approval_rate_fed_sponsors = (unique_drugs.groupby('Total FED Sponsors per Drug')['Is Approved'].mean().reset_index(name = 'Approval Rate'))
approval_rate_healthcare_sponsors = (unique_drugs.groupby('Total HOSPITALS Sponsors per Drug')['Is Approved'].mean().reset_index(name = 'Approval Rate'))
approval_rate_industry_sponsors = (unique_drugs.groupby('Total INDUSTRY Sponsors per Drug')['Is Approved'].mean().reset_index(name = 'Approval Rate'))
approval_rate_network_sponsors = (unique_drugs.groupby('Total NETWORK Sponsors per Drug')['Is Approved'].mean().reset_index(name = 'Approval Rate'))
approval_rate_nih_sponsors = (unique_drugs.groupby('Total NIH Sponsors per Drug')['Is Approved'].mean().reset_index(name = 'Approval Rate'))
approval_rate_other_sponsors = (unique_drugs.groupby('Total OTHER Sponsors per Drug')['Is Approved'].mean().reset_index(name = 'Approval Rate'))
approval_rate_top10pharma_sponsors = (unique_drugs.groupby('Total TOP 10 PHARMA Sponsors per Drug')['Is Approved'].mean().reset_index(name = 'Approval Rate'))

In [205]:
# Create subplots of bar charts for each feature and combine them in a single plot to investigate the drug approval rate
fig11 = make_subplots(rows = 3, cols = 3, subplot_titles=('Number of Academic Sponsors per Drug', 'Number of Cancer Center Sponsors per Drug', 'Number of FED Sponsors per Drug', 
                                                         'Number of Healthcare Institution per Drug', 'Number of Industry Sponsors per Drug', 'Number of Network Sponsors per Drug', 
                                                         'Number of NIH Sponsors per Drug', 'Number of Other Sponsors per Drug', 'Number of Top 10 Pharma Sponsors per Drug'), 
                                                         vertical_spacing = 0.08, horizontal_spacing = 0.03)

fig11.add_trace(go.Bar(x = approval_rate_academic_sponsors['Total ACADEMIC Sponsors per Drug'], y = approval_rate_academic_sponsors['Approval Rate'], 
                      hovertemplate = 'No. of Academic Sponsors: %{x} <br>Approval Rate: %{y:.2f} <extra></extra>'), row = 1, col = 1)
fig11.add_trace(go.Bar(x = approval_rate_cancercenter_sponsors['Total CANCER CENTERS Sponsors per Drug'], y = approval_rate_cancercenter_sponsors['Approval Rate'], 
                      hovertemplate = 'No. of Cancer Center Sponsors: %{x} <br>Approval Rate: %{y:.2f} <extra></extra>'), row = 1, col = 2)
fig11.add_trace(go.Bar(x = approval_rate_fed_sponsors['Total FED Sponsors per Drug'], y = approval_rate_fed_sponsors['Approval Rate'], 
                      hovertemplate = 'No. of FED Sponsors: %{x} <br>Approval Rate: %{y:.2f} <extra></extra>'), row = 1, col = 3)   
fig11.add_trace(go.Bar(x = approval_rate_healthcare_sponsors['Total HOSPITALS Sponsors per Drug'], y = approval_rate_healthcare_sponsors['Approval Rate'], 
                      hovertemplate = 'No. of Healthcare Institution Sponsors: %{x} <br>Approval Rate: %{y:.2f} <extra></extra>'), row = 2, col = 1)
fig11.add_trace(go.Bar(x = approval_rate_industry_sponsors['Total INDUSTRY Sponsors per Drug'], y = approval_rate_industry_sponsors['Approval Rate'], 
                      hovertemplate = 'No. of Industry Sponsors: %{x} <br>Approval Rate: %{y:.2f} <extra></extra>'), row = 2, col = 2)
fig11.add_trace(go.Bar(x = approval_rate_network_sponsors['Total NETWORK Sponsors per Drug'], y = approval_rate_network_sponsors['Approval Rate'], 
                      hovertemplate = 'No. of Network Sponsors: %{x} <br>Approval Rate: %{y:.2f} <extra></extra>'), row = 2, col = 3)
fig11.add_trace(go.Bar(x = approval_rate_nih_sponsors['Total NIH Sponsors per Drug'], y = approval_rate_nih_sponsors['Approval Rate'], 
                      hovertemplate = 'No. of NIH Sponsors: %{x} <br>Approval Rate: %{y:.2f} <extra></extra>'), row = 3, col = 1)
fig11.add_trace(go.Bar(x = approval_rate_other_sponsors['Total OTHER Sponsors per Drug'], y = approval_rate_other_sponsors['Approval Rate'], 
                      hovertemplate = 'No. of Other Sponsors: %{x} <br>Approval Rate: %{y:.2f} <extra></extra>'), row = 3, col = 2)
fig11.add_trace(go.Bar(x = approval_rate_top10pharma_sponsors['Total TOP 10 PHARMA Sponsors per Drug'], y = approval_rate_top10pharma_sponsors['Approval Rate'], 
                      hovertemplate = 'No. of Top 10 Pharma Sponsors: %{x} <br>Approval Rate: %{y:.2f} <extra></extra>'), row = 3, col = 3)

fig11.add_annotation(text = 'Drug Approval Rate', xref = 'paper', yref = 'paper', x = -0.047, y = 0.5, showarrow = False, textangle = -90, font = dict(size = 20))  
fig11.update_layout(title = {'text' : 'Drug Approval Rates for Different Types of Sponsors', 'y' : 0.98, 'x' : 0.5, 'xanchor' : 'center', 'yanchor' : 'top', 'font' : {'size' : 24}}, 
                   margin = dict(l = 80, r = 40, t = 80, b = 50), showlegend = False, height = 1000)

A positive correlation was observed between the no. of FED sponsosrs and drug approval rate. For all other sponsor types, the correlation appears quite erratic. <br>

However, an important point to note is that once the number of sponsors exceeds 40, drug approval rate is almost 100%. This is likely because more trials are being sponsored for the same drug which increases the likelihood of the drug being approved.

In [ ]:
# Group by the no. of medical conditions tested per drug
approval_rate_drug_condition = (unique_drugs.groupby('No. of Medical Conditions Tested per Drug')['Is Approved'].mean().reset_index(name = 'Approval Rate'))

In [207]:
fig12 = px.bar(approval_rate_drug_condition,
              x = 'No. of Medical Conditions Tested per Drug',
              y = 'Approval Rate',
              width = 750, 
              height = 500)
fig12.update_layout(title = {'text' : 'Drug Approval Rate VS Number of Medical Conditions Tested', 'x' : 0.5, 'xanchor' : 'center'}, xaxis_title = 'No. of Medical Conditions Tested per Drug', yaxis_title_text='Approval Rate')
fig12.update_traces(hovertemplate ='%{x}<br>Average Trial Duration: %{y:.2f} yrs<br><extra></extra>')
fig12.show()

Drug approval rate increases for drugs being tested for more medical conditions likely because it is much easier to get approval for a drug that is widely studied and tested for multiple different conditions.

In [ ]:
trial_drugbank_df.to_csv('./Datasets/Model Training Data.csv')
unseen_trial_drugbank_df.to_csv('./Datasets/Model Unseen Data.csv')

Exporting of clean data to be used for classifier model.